## Роевой интеллект: частицы

Реализую алгоритм роя частиц (Particle Swarm Optimization, PSO) —
метод оптимизации, вдохновлённый коллективным поведением стай птиц и косяков рыб.
В отличие от муравьиного алгоритма, где агенты обмениваются информацией
через феромоны, здесь частицы (агенты) взаимодействуют друг с другом
напрямую, обновляя свои скорости и позиции на основе как собственного опыта,
так и успехов соседей.

Каждая частица помнит лучшее решение, которое она нашла,
а также знает лучшее решение в своём окружении (или во всей популяции,
в зависимости от варианта алгоритма). Практическая задача — минимизация сложной
математической функции с множеством локальных минимумов. Частицы «летают»
по пространству решений, со временем сближаясь к глобальному минимуму.

Для визуализации я использовал двухмерную функцию с рельефной поверхностью,
по которой можно наблюдать, как частицы постепенно сходятся к наиболее
выгодной точке. Я также визуализировал траектории движения частиц, чтобы
отследить, насколько сильно на них влияют собственные открытия и находки других.

In [58]:
%use kandy

## Частица

In [59]:
data class Particle(
  var position: MutableList<Double>,
  var velocity: MutableList<Double>,
  var bestPosition: MutableList<Double>,
  var bestFitness: Double
)

## Функция Растринга

Немного переделал. Wiki: https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%A0%D0%B0%D1%81%D1%82%D1%80%D0%B8%D0%B3%D0%B8%D0%BD%D0%B0

In [60]:
fun rastrigin(position: List<Double>): Double {
  return position.size + position.sumOf { x -> x * x - 5_000 * cos(x / 2.25) }
}

plot {
  line {
    val keys = (-100..100).toList()

    x(keys)
    y(keys.map { rastrigin(listOf(it.toDouble())) })
  }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="VFcjDf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[-100.0,-99.0,-98.0,-97.0,-96.0,-95.0,-94.0,-93.0,-92.0,-91.0,-90.0,-89.0,-88.0,-87.0,-86.0,-85.0,-84.0,-83.0,-82.0,-81.0,-80.0,-79.0,-78.0,-77.0,-76.0,-75.0,-74.0,-73.0,-72.0,-71.0,-70.0,-69.0,-68.0,-67.0,-66.0,-65.0,-64.0,-63.0,-62.0,-61.0,-60.0,-59.0,-58.0,-57.0,-56.0,-55.0,-54.0,-53.0,-52.0,-51.0,-50.0,-49.0,-48.0,-47.0,-46.0,-45.0,-44.0,-43.0,-42.0,-41.0,-40.0,-39.0,-38.0,-37.0,-36.0,-35.0,-34.0,-33.0,-32.0,-31.0,-30.0,-29.0,-28.0,-27.0,-26.0,-25.0,-24.0,-23.0,-22.0,-21.0,-20.0,-19.0,-18.0,-17.0,-16.0,-15.0,-14.0,-13.0,-12.0,-11.0,-10.0,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,-1.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0],
"y":[5525.514273612494,4802.783456761544,5053.403720154746,6190.402150611489,7954.970572267207,9966.752146288978,11797.74495618415,13055.4630632048,13459.196783790445,12894.554766712148,11435.69030826131,9330.89331774395,6954.347423912866,4733.42584845996,3065.6525064273183,2241.462846638853,2387.772051934161,3443.315335216024,5170.551728096623,7201.818448137024,9110.768020603848,10495.207874745984,11055.246626254311,10651.563171545778,9332.476870320212,7324.5590620178555,4988.610104738519,2749.5556270720676,1013.8831974674486,90.6615208802059,131.4881850179545,1101.0341487999885,2783.9081984017594,4826.5076025352955,6805.723356969751,8311.150543847794,9024.830278618869,8783.029331567832,7608.0536026225345,5703.914070247291,3416.6873234185114,1167.2726515321915,-630.3889658588187,-1649.7438651038528,-1715.0752912457883,-835.6452797076263,796.1049633150151,2841.946514475688,4883.5808367480895,6503.915461783588,7368.105332327293,7288.613418269665,6261.649285611751,4467.905580297729,2237.4493276190674,-14.410309066959826,-1867.8168734142555,-2979.9450031994884,-3151.6116249063843,-2365.9773566262006,-791.818411098172,1249.266734034476,3345.344133097864,5074.183410436597,6085.297401616924,6168.042041247233,5292.544644178757,3615.508187984276,1449.7638207908321,-796.5124376710173,-2699.108566620418,-3900.2012203877475,-4177.881041100469,-3489.269793662461,-1978.8516351878125,49.60025227124777,2192.0469918768995,4022.689032959037,5176.699285284133,5421.109263941289,4700.07502001831,3145.7276568039733,1052.5001690430677,-1180.0810826458205,-3125.024607838169,-4410.838515254876,-4793.711632898777,-4204.886180001661,-2764.017449295853,-755.9247045103975,1424.7493909174145,3350.21810431806,4642.669878835238,5047.677368763108,4483.632841065207,3057.604611869177,1044.533601840789,-1166.187866514947,-3146.3752546147257,-4512.248346794493,-4999.0,-4512.248346794493,-3146.3752546147257,-1166.187866514947,1044.533601840789,3057.604611869177,4483.632841065207,5047.677368763108,4642.669878835238,3350.21810431806,1424.7493909174145,-755.9247045103975,-2764.017449295853,-4204.886180001661,-4793.711632898777,-4410.838515254876,-3125.024607838169,-1180.0810826458205,1052.5001690430677,3145.7276568039733,4700.07502001831,5421.109263941289,5176.699285284133,4022.

## Параметры задачи и PSO

In [61]:
val dimensions = 2
val particleCount = 30
val iterations = 100
val lowerBound = -5.12
val upperBound = 5.12

val inertiaWeight = 0.7
val cognitiveComponent = 1.5  // личный опыт (c1)
val socialComponent = 1.5     // коллективный опыт (c2)

## Инициализация роя

In [62]:
val swarm = List(particleCount) {
  val position = List(dimensions) { Random.nextDouble(lowerBound, upperBound) }.toMutableList()
  val velocity = List(dimensions) { Random.nextDouble(-1.0, 1.0) }.toMutableList()
  val fitness = rastrigin(position)
  Particle(position, velocity, position.toMutableList(), fitness)
}

plot {
  points {
    x(swarm.map { it.position[0] })
    y(swarm.map { it.position[1] })
  }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Zu4BBR"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[4.314772136567007,1.9520319320731074,0.7857754480955519,3.5553723111907525,4.448912487795369,0.3437553150782291,2.936276690775407,-1.252743265505536,-1.652696085458763,-4.93472420376149,3.275042776116721,-3.757422967688774,5.011025225138343,-2.538626214082336,-3.1214059350550203,-4.290363325036574,-4.135276723492134,4.613092442180879,-2.421878293975548,-4.647947024795333,-1.1503252875531702,1.4585247193834645,1.9252203880081247,0.19700915767006943,0.3777374025663214,-5.081767322741242,0.7985953899485363,-0.6505610328437088,0.6941105729731802,3.761738607940978],
"y":[4.477487237518081,-1.6646189905651476,1.3233404796303851,-1.5320584966471755,2.6611853621674753,0.22048558552519815,-2.7519131396054637,-0.41371893421968853,2.9042376274698407,-3.681157568155095,-2.7144754655437566,4.6215158726774535,1.5121128907828414,-2.726483908136726,-4.407171822397615,-5.039560416958513,-3.1188783035585788,-4.094665735652877,0.3008528874192269,4.443306543021904,0.6572663905185383,-4.462113383347595,3.4013674867373043,1.702008260315747,-1.1115018818778841,2.1049723600944796,1.1641795599663851,-4.724840043110145,-3.846514759323412,-1.0916293766171803]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"float",
"column":"y"
}]
}
}],
"spec_id":"101"
};
 var containerDiv = document.getElementById("Zu4BBR");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 
 
 
 
 -2 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 y 
 
 
 
 
 x

In [63]:
import kotlin.math.*

var globalBest = swarm.minBy { it.bestFitness }.bestPosition.toMutableList()
var globalBestFitness = rastrigin(globalBest)

// Лог для визуализации
val trajectoryLog = mutableListOf<Triple<Int, Double, Double>>() // (iter, x, y)

for (iter in 0 until iterations) {
  for (particle in swarm) {
    // Обновить скорость и позицию
    for (d in 0 until dimensions) {
      val r1 = Math.random()
      val r2 = Math.random()
      val cognitive = cognitiveComponent * r1 * (particle.bestPosition[d] - particle.position[d])
      val social = socialComponent * r2 * (globalBest[d] - particle.position[d])
      particle.velocity[d] = inertiaWeight * particle.velocity[d] + cognitive + social
      particle.position[d] += particle.velocity[d]
      particle.position[d] = particle.position[d].coerceIn(lowerBound, upperBound)
    }

    val fitness = rastrigin(particle.position)
    if (fitness < particle.bestFitness) {
      particle.bestFitness = fitness
      particle.bestPosition = particle.position.toMutableList()
    }

    if (fitness < globalBestFitness) {
      globalBestFitness = fitness
      globalBest = particle.position.toMutableList()
    }

    if (particle == swarm[0]) {
      trajectoryLog.add(Triple(iter, particle.position[0], particle.position[1]))
    }
  }
}

In [64]:
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf

val x = trajectoryLog.map { it.second }
val y = trajectoryLog.map { it.third }
val step = trajectoryLog.map { it.first.toDouble() }

val data = dataFrameOf("x" to x, "y" to y)

plot(data) {
  path {
    x("x")
    y("y")
    width = 2.0
    color = Color.hex("#4682B4")
  }

  points {
    x("x")
    y("y")
    size = 4.0
    color = Color.RED
  }

  x.axis.name = "X"
  y.axis.name = "Y"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="k67l5h"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"x":[-1.1997016760526131,-4.137365391076958,-3.9416259154571565,1.5425431763215487,0.9664477276507761,-0.9747852172714,-2.0185693669432547,0.23582286203561864,1.1525140653024004,0.43056271592014816,-0.4689232073615548,-0.8433114286502545,0.31415026484423714,0.9606207002968812,0.5978517106377135,-0.7517331935915939,0.5314604353495127,0.7105475625294169,0.26479132870138933,-0.032564942760678284,0.2196608661796316,0.21979366146245333,0.20427625988772757,0.0653151939559781,0.14939596344354703,0.22422476479509187,-0.0068285453963855824,0.0014713187778614516,0.28298879795086934,0.04508262899775739,0.009458099087990168,0.17106324374731902,0.16520390648284894,0.14302367238141273,0.08104521783833615,-0.023578879130265934,0.009259056760215406,0.0917869634617863,0.09282134241988556,0.03740202934256769,-0.005845198824933832,-0.009051597644116963,0.075407264478996,0.09473926684856324,-0.04180812555340305,0.030903658940373266,0.055172195817951325,0.032657920984585496,-5.973382507083319E-4,-0.02316786854149991,-0.028045899642423863,0.005290169408415292,0.019812445954645305,-0.017543256598363574,-0.004666558644545189,0.008465246044534363,0.015352234064787609,0.01581502643128285,0.0013456821152372928,-0.010317117416235757,-0.009844229522808265,0.0034990828391727267,0.005594714391334995,2.860709233688868E-4,-0.003875553250580752,-0.0022387906899272096,0.0022469040610593976,0.001630559879506182,-2.49126105896577E-4,-0.0011088904698480682,8.887998825903712E-4,0.0014613512411000102,2.5251894973951227E-4,-7.329098464157696E-4,-9.638081345280753E-4,7.955481028032838E-4,8.712355643112192E-4,-0.0010498090710063755,-4.3627640220384996E-4,0.0012867462770650515,6.093400315295269E-4,-4.6344415975259736E-4,-6.001602048801195E-4,-4.3040137382711755E-4,3.2052628639435584E-4,6.017270446931478E-4,4.362195424137713E-4,5.8885330460814586E-5,-2.845644588995094E-4,3.3794197779213114E-5,2.2425384779321185E-4,-5.480420244558673E-5,-2.248032072276128E-4,2.1096190209576575E-4,1.4418588915568407E-4,-2.2118184015508703E-5,-1.2870771633914624E-4,-1.5539700646921467E-4,-1.1625031983896721E-4,-1.465161838363715E-6],
"y":[-0.0512223184802183,-3.3971542831967843,-2.008725848016084,2.3952791582020616,1.9843733236926198,-2.9963268353965287,-1.1431844248897067,2.6960618953266295,4.734635743326762,-5.12,-0.8915166848194751,2.1634210196129744,-0.3009909086940836,-1.9382775045050287,-2.0770744645945336,-1.5416210021915884,0.31409164538062573,0.995781567183449,0.6289637054378778,-1.1931206807086534,0.08559086728924825,0.8830665310238417,0.5315580140991827,-0.6342243600844246,-0.24159903351186474,0.3984624754968227,0.7538915836303286,-0.8399544490606687,-1.025938240085088,-0.4737208109326382,0.4590998615932642,0.8800586566463673,0.216788009978369,-0.39624334717082577,-0.05483226613177827,0.25553808859449106,0.2116333657859689,-0.3076812759804597,-0.21950941166054974,0.16219655003509179,0.19346936607301263,-0.10649297373363797,-0.2556560607453797,0.027969193345841747,0.145440305769183,0.08948931087473828,0.015427945278060656,-0.048361708554860064,-0.01885547042135837,0.007814996162372332,0.0012028112066800496,-0.02401454158145972,-0.006676765872612393,-0.008339869406930996,-0.014254813444075597,-0.0011267773000541992,0.008499550076190639,-0.002450601794620186,-0.008175255790779542,-0.0014686409260569015,0.0035892990768085454,-0.013553198721985349,-0.014353977447214972,-1.5220645224275388E-4,0.009991035986986767,0.006937075241094952,-0.00312301511543874,-0.0029918491109414925,9.422666500405923E-4,0.001957947606457325,0.0011107100125473338,1.314443617149

Для меня тут нет "интеллекта", скорее моделирование физического процесса. Запускаем на кривую непрерывную поверхность кучу шариков и смотрим куда больше упало.